### Imports

In [103]:
import PIL
import glob #allows us to access every image in folder
import re #allows us to extract emotion from file path
import numpy as np

from PIL import Image #allows us to resize images/get their initial pixel value
from sklearn import preprocessing #allows us to normalize our data



# a.i

### Resizing images & Converting to np array

Start by seeing what the initial pixel size is, because we want to get it down to a 30x30 image

In [93]:
def get_num_pixels(filepath):
    width, height = Image.open(filepath).size
    return width, height

In [106]:
width_1_happy, height_1_happy = get_num_pixels("Images/subject01.happy.gif")

print("Width: ", width_1_happy)
print("Height: ", height_1_happy)

Width:  320
Height:  243


Now we know all images have a size of 320x243 (even when their file sizes are different)... this will NOT resize in a way that keeps the aspect ratio in tact. Each image is surrounded by large portions of whitespace on the left or right, so we will chop that down

#### Changing the size of the images

Because we're unable to maintain ratios and shrink down to a 30x30 size, we're going to chop off some of the extra white space on the images widths. We're going to get each image down to 243x243 so when we shrink to 30x30 nothing gets stretched out of proportion

In [95]:
list_subjects = []

for subject_num in range(1,16):
    if subject_num < 10:
        list_subjects.append("subject0" + str(subject_num))
    else:
        list_subjects.append("subject" + str(subject_num))


This function takes in a list of people whose image needs to be cropped in a similar fashion, and the dimensions of their cropping.

It then crops it into a square so it can be resized into a 30x30 shape without any warping

The image is then converted into a np array of its pixel values & added to our training data set. The label for each image is also added to an array (this is in the same order as our training data)

In [111]:
def image_resize(list_subjects, left, right, top, bottom):
    subject_labels = []
    training_data = []
    
    for subject in list_subjects:
        for image_path in glob.glob("Images/" + subject + "*"):
            #for each image, extract the label
            search_string = subject + ".(.+?).gif"
            m = re.search(search_string, image_path)
            if m:
                emotion = m.group(1)
                subject_labels.append(subject + ": " + emotion)
            
            #for each image, trim off the specified edges & save to new folder
            crop_image_path = "Images_Cropped/" + subject + "." + emotion + ".gif"
            orig_image = Image.open(image_path)
            orig_image.crop((left, top, right, bottom)).save(crop_image_path)
            
            #for each cropped image, resize to 30x30
            resize_image_path = "Images_Resized/" + subject + "." + emotion + ".gif"
            basewidth = 30
            crop_image = Image.open(crop_image_path)
            wpercent = (basewidth / float(crop_image.size[0]))
            hsize = int((float(crop_image.size[1]) * float(wpercent)))
            resize_image = crop_image.resize((basewidth, hsize), PIL.Image.ANTIALIAS)
            resize_image.save(resize_image_path)
            
            #for each resized image, convert to 1x90 np array
            image_frame = Image.open(resize_image_path)
            np_frame = np.array(image_frame.getdata()) #gets pixel values
            image_list = list(np_frame)
            training_data.append(image_list)

    return subject_labels, training_data 
        

This organizes the subjects based on their cropping dimension needs. It's kind of a hack job, but it works

In [112]:
first_list = list_subjects[0:5]

second_list = list_subjects[5:7]
second_list.append(list_subjects[13])

third_list = list_subjects[7:10]
third_list.append(list_subjects[12])
third_list.append(list_subjects[14])

fourth_list = list_subjects[10:12]

This creates an overarching list for all training data and labels, and populates it with the ouput of our image resize function

In [113]:
temp_training_data = []
temp_labels = []

first_labels, first_data = image_resize(first_list, 54, 297, 0, 243)
temp_labels.append(first_labels)
temp_training_data.append(first_data)

second_labels, second_data = image_resize(second_list, 0, 243, 0, 243)
temp_labels.append(second_labels)
temp_training_data.append(second_data)

third_labels, third_data = image_resize(third_list, 43, 286, 0, 243)
temp_labels.append(third_labels)
temp_training_data.append(third_data)

fourth_labels, fourth_data = image_resize(fourth_list, 69, 312, 0, 243)
temp_labels.append(fourth_labels)
temp_training_data.append(fourth_data)

Get rid of nested sublists and convert training data to np array

In [114]:
labels = []
for sublist in temp_labels:
    for item in sublist:
        labels.append(item)

training_data = []
for sublist in temp_training_data:
    for item in sublist:
        training_data.append(item)

training_data = np.array(training_data)

Let's verify we have 165 vectors of size 1x900 for our training data

And an array of size 65 for our labels

In [115]:
print(len(labels))
print(training_data.shape)

165
(165, 900)


Below shows how the image is now represented by numbers. You cannot see it here because each row is broken up in the middle, but if you copy and past the output below into a text file and give each row its own line you will see the outline of a face :)

In [117]:
print(str(labels[2])) #Prints letter being shown
#Reshape image to be 14x9, then loop through each new row to print
for row in training_data[2].reshape(30, 30):
    print(row)

subject01: noglasses
[213 213 213 213 213 213 213 213 213 213 213 213 213 213 213 213 213 213
 213 213 213 213 213 213 213 213 213 213 213 213]
[213 213 213 213 213 213 213 213 213 213 213 213 213 213 213 213 213 213
 213 213 213 213 213 213 213 213 213 213 213 213]
[213 213 213 213 213 213 213 213 213 213 213 213 213 213 213 206 198 120
  98 113 143 213 213 213 213 213 213 213 213 213]
[213 213 213 213 213 213 213 213 213 213 213 213 213 208 139  80  64  56
  52  91 100 122 119 213 213 213 213 213 213 213]
[213 213 213 213 213 213 213 213 213 213 213 178  67 101  62  69  37  39
  81 102 105  89  78  91 213 213 213 213 213 213]
[213 213 213 213 213 213 213 213 213 213 104 120  74  74  60  66  50  30
  62  98 113  79  79 101  76 173 213 213 213 213]
[213 213 213 213 213 213 213 213 213 119 126 139 104  79  88 153 111  27
  31  83  89 110  78  71  76  70 191 213 213 213]
[213 213 213 213 213 213 213 213 129 101  84  56  80 147 144 101 112  72
  41  65  86  92  81  78  59  49  70 213 213 

### Normalized Training Data

Converts values to the range of 0:1

In [104]:
normalized_training_data = preprocessing.normalize(training_data)